In [1]:
import numpy as np
from qore import Mine, ASP, QAOA, VQE, Pseudoflow, PseudoflowSolver, ASPSolver

In [2]:
qmine = Mine('mine_config_real.txt')
qmine.plot_mine()

+---+--------+-------+--------+--------+--------+--------+
|   |   0    |   1   |   2    |   3    |   4    |   5    |
+---+--------+-------+--------+--------+--------+--------+
| 0 | 0.500  | 1.500 | -0.500 | -0.500 | -0.500 | -1.000 |
| 1 | 0.500  | 1.500 | 0.500  | 1.000  | -1.000 | -2.000 |
| 2 | -0.500 | 2.500 | 2.500  | -0.500 | -1.500 | -0.500 |
| 3 | -3.000 | 2.000 | 3.000  | -1.000 | -2.000 | -1.000 |
| 4 | -2.000 | 1.000 | 4.000  | 1.000  | 0.500  | 2.000  |
| 5 | 1.000  | 1.000 | 0.500  | 4.000  | -2.000 | 1.500  |
+---+--------+-------+--------+--------+--------+--------+
/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/quantum_info/operators/symplectic/pauli.py:185: DeprecationWarning: Initializing Pauli from `Pauli(z=z, x=x)` kwargs is deprecated as of version 0.17.0 and will be removed no earlier than 3 months after the release date. Use tuple initialization `Pauli((z, x))` instead.
  base_z, base_x, base_phase = self._from_array_deprecated(z, x)


In [3]:
pf = PseudoflowSolver(qmine)

In [4]:
res = pf.solve()

In [5]:
res

{'opt_config': '101000111100111110111111111111111111'}

In [6]:
qmine.plot_mine_state(res['opt_config'])

+---+---+---+---+---+---+---+
|   | 0 | 1 | 2 | 3 | 4 | 5 |
+---+---+---+---+---+---+---+
| 0 | 1 | 1 | 1 | 1 | 1 | 1 |
| 1 | 1 | 1 | 1 | 1 | 1 | 1 |
| 2 | 1 | 1 | 1 | 1 | 1 | 1 |
| 3 | 0 | 1 | 1 | 1 | 1 | 1 |
| 4 | 0 | 0 | 1 | 1 | 1 | 1 |
| 5 | 0 | 0 | 0 | 1 | 0 | 1 |
+---+---+---+---+---+---+---+


In [7]:
idx2frag = [0]*qmine.nqubits
frag2idx = {0:[], 1:[], 2:[]}
for i in range(qmine.nqubits):
    frag = qmine.idx2cord[i][0] // 2
    idx2frag[i] = frag
    frag2idx[frag].append(i)

In [8]:
from qore.utils import null_operator, z_projector

H_p_inner = [null_operator(12)]*3
H_s_inner = [null_operator(12)]*3

In [9]:
null_operator(12).num_qubits

Operator is empty, Return 0.


0

In [39]:
H_p_inner[0].num_qubits

Operator is empty, Return 0.


0

In [35]:
for f in range(len(frag2idx)):
    for ii, i in enumerate(frag2idx[f]):
        print(ii, i)
        H_p_inner[f] += qmine.dat[qmine.idx2cord[i]]*z_projector(1, ii, H_p_inner[f].num_qubits)

Operator is empty, Return 0.
0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
0 12


IndexError: list assignment index out of range

In [19]:
asp = ASPSolver(qmine)

/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_class('aqua.QuantumInstance',


In [20]:
penalty = 10.0
evol_time = 20
nsteps = 200

In [21]:
res = asp.solve(penalty=penalty, evol_time=evol_time, nsteps=nsteps)

/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/aqua/operators/legacy/weighted_pauli_operator.py:906: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc = QuantumCircuit() + state_in
/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)


In [12]:
res

{'opt_config': '1111',
 'opt_config_prob': 0.9287109375,
 'profit_avg': 6.753832133010106,
 'violation_avg': 8.87761596793435e-05,
 'ground_state': {'0111': 0.0048828125,
  '0010': 0.0009765625,
  '0110': 0.0654296875,
  '1111': 0.9287109375}}